__Find AQ__

In [1]:
import pandas as pd
import requests
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from time import sleep
import time

In [2]:
cd D:\1 RA\re_find_1104\save

D:\1 RA\re_find_1104\save


In [3]:
df = pd.read_stata("unique_coordinates_all_aq.dta")
df

,city,State,lat,lon,id_coordinate
0,Achalpur,Maharashtra,21.257400,77.509200,1.0
1,Agra,Uttar Pradesh,27.176200,77.980000,2.0
2,Ahiri,Maharashtra,19.412900,80.005400,3.0
3,Ahmadabad,Gujarat,23.020200,72.579700,4.0
4,Ahmadnagar,Maharashtra,19.110300,74.742700,5.0
...,...,...,...,...,...
592,Malapuram,Kerala,11.061862,76.068392,593.0
593,Pudukottai,Tamil Nadu,10.383287,78.800129,594.0
594,Tanjavur,Tamil Nadu,10.752820,79.131449,595.0
595,Tehri-Garhwal,Uttaranchal,30.385390,78.469057,596.0


#### __Test__

1.1 Test the API

In [4]:
lat = df.iloc[0, 2]
lon = df.iloc[0, 3]

In [5]:
lon

77.5092

In [6]:
lat

21.2574

In [7]:
developer_id = "c0e1278c4135f377f34d4403e99491b3"

In [8]:
def get_aq_post_request(lat, lon):
    parameters = {'lat': "{}".format(lat), 'lon': "{}".format(lon), 
                 'start': '1606266000', 'end': '1609459200',
                 'appid': developer_id}
    base = "http://api.openweathermap.org/data/2.5/air_pollution/history?"
    try:
        response = requests.get(base, parameters)
    except Exception:
        print('Error......')
    answer = response.json()
#     print(answer['result']['location'])
    return answer

In [9]:
a = get_aq_post_request(lat, lon)

In [10]:
len(a['list'])

888

In [11]:
a['list']

[{'main': {'aqi': 5},
  'components': {'co': 423.91,
   'no': 0,
   'no2': 4.5,
   'o3': 63.66,
   'so2': 4.71,
   'pm2_5': 55.52,
   'pm10': 60.93,
   'nh3': 7.47},
  'dt': 1606266000},
 {'main': {'aqi': 5},
  'components': {'co': 440.6,
   'no': 0.07,
   'no2': 6.26,
   'o3': 60.08,
   'so2': 4.47,
   'pm2_5': 55.3,
   'pm10': 60.83,
   'nh3': 8.36},
  'dt': 1606269600},
 {'main': {'aqi': 5},
  'components': {'co': 453.95,
   'no': 0.69,
   'no2': 6.77,
   'o3': 61.51,
   'so2': 3.82,
   'pm2_5': 53.65,
   'pm10': 59.01,
   'nh3': 8.74},
  'dt': 1606273200},
 {'main': {'aqi': 5},
  'components': {'co': 427.25,
   'no': 0.73,
   'no2': 4.5,
   'o3': 80.11,
   'so2': 3.73,
   'pm2_5': 50.56,
   'pm10': 55.3,
   'nh3': 7.92},
  'dt': 1606276800},
 {'main': {'aqi': 5},
  'components': {'co': 420.57,
   'no': 0.55,
   'no2': 3.34,
   'o3': 95.84,
   'so2': 3.93,
   'pm2_5': 50.97,
   'pm10': 55.77,
   'nh3': 7.47},
  'dt': 1606280400},
 {'main': {'aqi': 5},
  'components': {'co': 400.54,


1.2 Test Unix Timestamp

In [12]:
epoch = 1606266000

In [13]:
time.strftime("%Y-%m-%d", time.gmtime(epoch))

'2020-11-25'

1.3 Function for Json Convertion

In [9]:
def json_to_df(json_ls):  # json_ls: answer['list']
    
    len_df = len(json_ls)  # Number of Obs in DataFrame result.
    
    result = pd.DataFrame(columns=['date', 'aqi', 'co', 'no', 'no2', 'o3', 'so2',
                                  'pm2_5', 'pm10', 'nh3'], 
                          index=np.arange(len_df))  # Generate an empty DataFrame
    
    for i_ in range(len_df):
        main_ = json_ls[i_]['main']  # main
        components_ = json_ls[i_]['components']  # components
        date = time.strftime("%Y-%m-%d", time.gmtime(json_ls[i_]['dt']))  # date

        result.iloc[i_, 0] = date  # date
        result.iloc[i_, 1] = main_['aqi']  # aqi
        result.iloc[i_, 2] = components_['co']  # co
        result.iloc[i_, 3] = components_['no']  # no
        result.iloc[i_, 4] = components_['no2']  # no2
        result.iloc[i_, 5] = components_['o3']  # o3
        result.iloc[i_, 6] = components_['so2']  # so2
        result.iloc[i_, 7] = components_['pm2_5']  # pm25
        result.iloc[i_, 8] = components_['pm10']  # pm10
        result.iloc[i_, 9] = components_['nh3']  # nh3
    
    return result

In [15]:
b = json_to_df(a['list'])
b

,date,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,2020-11-25,5,423.91,0,4.5,63.66,4.71,55.52,60.93,7.47
1,2020-11-25,5,440.6,0.07,6.26,60.08,4.47,55.3,60.83,8.36
2,2020-11-25,5,453.95,0.69,6.77,61.51,3.82,53.65,59.01,8.74
3,2020-11-25,5,427.25,0.73,4.5,80.11,3.73,50.56,55.3,7.92
4,2020-11-25,5,420.57,0.55,3.34,95.84,3.93,50.97,55.77,7.47
...,...,...,...,...,...,...,...,...,...,...
883,2020-12-31,5,594.14,0,9.77,50.78,7.03,127.55,135.91,11.15
884,2020-12-31,5,540.73,0,7.28,59.37,11.92,132.6,140.97,9.5
885,2020-12-31,5,494,0,5.74,67.23,20.98,136.78,145.62,7.35
886,2020-12-31,5,467.3,0,5.44,67.23,25.51,138.69,147.75,6.4


#### __Loop__

In [10]:
cd D:\1 RA\re_find_1104\save\aq_get

D:\1 RA\re_find_1104\save\aq_get


Total Number of Locations

In [11]:
num, _ = df.shape
num

597

Set Start Point for Loop

In [12]:
start = 7  # Number of exisiting files in the folder *aq_get*
end = num

In [ ]:
%%time
for j_ in range(start, end):
    
    print(j_)
    
    lat = df.iloc[j_, 2]  # Latitude of current Obs.
    lon = df.iloc[j_, 3]  # Longitude of current Obs.
    id_coordinate = int(df.iloc[j_, 4])  # Id of Obs.
    temp_json = get_aq_post_request(lat, lon)
    print('Loading......')
    res_ = json_to_df(temp_json['list'])  # Temp DateFrame
    res_['id_coordinate'] = id_coordinate
    res_.to_excel('{}.xlsx'.format(id_coordinate), index=None)
    sleep(6)

7
Check Point......
8
Check Point......
9
Check Point......
10
Check Point......
11
Check Point......
12
Check Point......
13
Check Point......
14
Check Point......
15
Check Point......
